In [1]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

/home/sebastian/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#---------------------------------------------------------
# DELETE TRASH COLUMNS
#---------------------------------------------------------
def deleteTrashColumns(df):
    columns = list(df.columns.values)
    for column in columns:
        if "unnamed" in column.lower():
            df.drop(column, axis = 1, inplace = True) 
    return df
#---------------------------------------------------------
# PROCESS TRAIN SET
#---------------------------------------------------------
def ProcessTrainSet(df, withPrice):
    df.dropna(axis=0, how='any', subset=list(df.columns.values), inplace=True)
    
    target = df['price_aprox_usd'].tolist()
    if withPrice:
        df.drop('price_aprox_usd', axis = 1, inplace = True)
    
    columns = list(df.columns.values)
    data = list(df.values)
    return data, target
#---------------------------------------------------------
# GET DATA AS LIST
#---------------------------------------------------------
def GetDataAsList(df):
    columns = list(df.columns.values)
    data = list(df.values)
    return data
#---------------------------------------------------------
# PRINT SCORE
#---------------------------------------------------------
def printScore(prediction, target):
    err = mean_squared_error(prediction, target)
    print "mean_squared_error: ", err
    digitsQuantity = len(str(err).split('.')[0])
    print "digitsQuantity: ", digitsQuantity
#---------------------------------------------------------
# SAVE FINAL DF
#---------------------------------------------------------
def saveFinalDF(predictions, ids):
    aData = {'price_usd': predictions}
    final = pd.DataFrame(data = aData, index=ids)
    final.to_csv("properati_dataset_sample_submision.csv", \
    index = True, header = True, sep = ',', encoding = 'utf-8-sig')
    return final
#---------------------------------------------------------
# FIT AND GET INFO
#---------------------------------------------------------
def fitAndGetInfo(model, xTrain, yTrain, xTest, yTest):
    model.fit(xTrain, yTrain)
    prediction = model.predict(xTest)
    print "precision test: ", model.score(xTest, yTest)
    print "best params: ", model.best_params_
    print "best score: ", model.best_score_ 
    print "best estimator: ", model.best_estimator_
    printScore(prediction, yTest)

# Set de entrenamiento

In [3]:
df = pd.read_csv("dataTrain.csv", low_memory = False)
df = deleteTrashColumns(df)
dataTrain, targetTrain = ProcessTrainSet(df, True)

In [4]:
columns = list(df.columns.values)
print "data size: ", len(dataTrain)
print "target size: ", len(targetTrain)
print "columns size: ", len(columns)

data size:  248616
target size:  248616
columns size:  7


# Set de test

In [5]:
predictDf = pd.read_csv("dataTest.csv", low_memory = False)
ids = predictDf['id']
predictDf.drop('id', axis = 1, inplace = True)
predictDf = deleteTrashColumns(predictDf)
data = GetDataAsList(predictDf)

In [6]:
columns = list(predictDf.columns.values)
print "data size: ", len(data)
print "df size: ", len(predictDf.index)
print "columns size: ", len(columns)
#row should be 14166

data size:  14166
df size:  14166
columns size:  7


In [7]:
columns

['property_type',
 'place_name',
 'state_name',
 'lat',
 'lon',
 'surface_total_in_m2',
 'surface_covered_in_m2']

In [33]:
#df.drop('place_name', axis = 1, inplace = True)3
#predictDf.drop('place_name', axis = 1, inplace = True)

# Split

In [8]:
xTrain, xTest, yTrain, yTest = train_test_split(dataTrain, targetTrain, test_size=0.5, random_state=0)

# Prediccion 

# Linear Regression

In [176]:
from sklearn.linear_model import LinearRegression

In [177]:
params = {
    'normalize': [True, False],
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [i for i in range(1, 10)]
}
LRGS = GridSearchCV(LinearRegression(), params, cv=3, n_jobs=5, verbose=1, scoring=None, refit=True)

In [26]:
fitAndGetInfo(LRGS, xTrain, yTrain, xTest, yTest)

In [36]:
lrPrediciton = lr.predict(data)
final = saveFinalDF(lrPrediciton, ids)

In [37]:
final.head()

,price_usd
id,
3632,-2.843368e+05
3633,-4.285213e+06
2263404,-5.631776e+05
2263405,-7.699427e+05
2263406,-6.938132e+05


# Transformacion no lineal a lineal

#  (Este metodo da precios demasiado altos y es peor que el lineal hecho en el paso anterior)

Si tomamos como ejemplo una función f que toma la forma :  f(x) = a + bx + cx²

La función f es no lineal en función de x pero si es lineal en función de los parámetros desconocidos a, b, y c. O visto de otra manera: podemos sustituir nuestras variables x por un array z tal que: z = [1, x, x²]. Con el que podríamos reescribir nuestra función f como f(z) = a z0 + bz1 + c*z2

Scikit-learn tiene un objeto PolynomialFeatures que nos va a servir para convertir nuestra variable x en un array z del tipo z = [1, x, x2, …, n^n], que es lo que nos interesa.

El resultado de esa transformación se la pasamos a nuestro modelo Ridge. Para facilitar la tarea en este tipo de casos —donde se realizan varios pasos que van desde el pre-tratamiento de los datos hasta un posible post-tratamiento pasando por el entrenamiento—, podemos hacer uso de las Pipeline que nos permiten encadenar multiples estimadores en uno. Esto es especialmente útil cuando hay secuencia de pasos predefinidos en el procesado de datos con, por ejemplo, selección de atributos, normalización y clasificación.

In [186]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
params = {
    'normalize': [True, False],
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [i for i in range(1, 10)]
}
PFGS = GridSearchCV(LinearRegression(), params, cv=3, n_jobs=5, verbose=1, scoring=None, refit=True)

In [ ]:
poly = PolynomialFeatures(degree = 2)
zTrain = poly.fit_transform(xTrain)
zTest = poly.fit_transform(xTest)

In [ ]:
fitAndGetInfo(PFGS, zTrain, yTrain, zTest, yTest)

In [189]:
DataTransformed = poly.fit_transform(data)
predictions = PFGS.predict(DataTransformed)
final = saveFinalDF(predictions, ids)

In [190]:
final.head()

,price_usd
id,
3632,-1.309150e+22
3633,-2.594522e+23
2263404,-1.635580e+22
2263405,-1.460626e+22
2263406,-1.452099e+22


# Decicion Tree

In [47]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [48]:
DecisionTree = DecisionTreeRegressor(random_state=103040)
DecisionTree.get_params()

{'criterion': 'mse',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 103040,
 'splitter': 'best'}

In [49]:
columns = list(predictDf.columns.values)
maxFeatures = len(columns)
params = {
    #'max_depth': [5, 15, 25, 35, 45, 55, 67, 76, 85, 95, 100],
    #'criterion': ["mse", "friedman_mse", "mae"]
    'max_features': [i for i in range(1, maxFeatures)],
    'random_state': [103040]
}
DTGS = GridSearchCV(DecisionTreeRegressor(), params, cv=10, n_jobs=1, verbose=1, scoring=None, refit=True)

In [50]:
fitAndGetInfo(DTGS, xTrain, yTrain, xTest, yTest)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   35.0s finished


precision test:  0.807585487921
best params:  {'max_features': 1, 'random_state': 103040}
best score:  0.930130376046
best estimator:  DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=1,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=103040, splitter='best')
mean_squared_error:  30020753695.5
digitsQuantity:  11


In [51]:
predictionDTR = DTGS.predict(data)
final = saveFinalDF(predictionDTR, ids)

In [53]:
final
#3632: $410000

,price_usd
id,
3632,500000.00
3633,120000.00
2263404,89000.00
2263405,46762.47
2263406,65000.00
2263407,46762.47
2263408,46762.47
2263409,46762.47
2263410,55000.00


# Random Forest Regressor

In [54]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [55]:
RandomForest = RandomForestRegressor(random_state=103040)
RandomForest.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 103040,
 'verbose': 0,
 'warm_start': False}

In [58]:
columns = list(predictDf.columns.values)
maxFeatures = len(columns)
params = {
    'max_depth': [5, 15, 25, 35, 45, 55, 67, 76, 85, 95, 100],
    #'min_samples_leaf': [3, 5, 9],
    'max_features': [i for i in range(1, maxFeatures)],
    'random_state': [103040]
}
RFGS = GridSearchCV(RandomForestRegressor(), params, cv=10, n_jobs=1, verbose=1, scoring=None, refit=True)

In [59]:
fitAndGetInfo(RFGS, xTrain, yTrain, xTest, yTest)

Fitting 10 folds for each of 66 candidates, totalling 660 fits


[Parallel(n_jobs=1)]: Done 660 out of 660 | elapsed: 29.0min finished


precision test:  0.831443491171
best params:  {'max_features': 1, 'random_state': 103040, 'max_depth': 45}
best score:  0.945418759845
best estimator:  RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=45,
           max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=103040,
           verbose=0, warm_start=False)
mean_squared_error:  26298398081.7
digitsQuantity:  11


In [60]:
RFPrediction = RFGS.predict(data)
final = saveFinalDF(RFPrediction, ids)

In [61]:
final

,price_usd
id,
3632,270900.000
3633,181292.200
2263404,143207.119
2263405,137441.519
2263406,180507.119
2263407,137441.519
2263408,137441.519
2263409,137441.519
2263410,150592.200
